# Προσέξτε το Κενό

---

> Πάνος Λουρίδας, Αναπληρωτής Καθηγητής <br />
> Τμήμα Διοικητικής Επιστήμης και Τεχνολογίας <br />
> Οικονομικό Πανεπιστήμιο Αθηνών <br />
> louridas@aueb.gr


Θα αναπαράξουμε τις αναπαραστάσεις του [Gapminder](http://www.gapminder.org/) για το ΑΕΠ και το προσδόκιμο ζωής, χρησιμοποιώντας την Python. Συγκεκριμένα, θα χρησιμοποιήσουμε το [pandas](http://pandas.pydata.org/) και το [bokeh](http://bokeh.pydata.org/en/latest/). Ο σκοπός είναι να δούμε πώς μπορούμε να κατασκευάσουμε αλληλεπιδραστικές παραστάσεις με ανοιχτά εργαλεία.

Αν σας ενδιαφέρει απλώς το τελικό αποτέλεσμα, πηγαίνετε στο τέλος.

Αλλιώς συνεχίστε παίρνοντας τα πράγματα με τη σειρά τους.

In [1]:
import pandas as pd
import numpy as np
import re

Θα διαβάσουμε τα δεδομένα του κατακεφαλήν ΑΕΠ, σε ισοδύναμη αγοραστική δύναμη, τα οποία τα έχουμε κατεβάσει από το Gapminder και τα έχουμε τοποθετήσει στον κατάλογο (φάκελλο) `gapminder`. Μπορείτε να βρείτε όλα τα δεδομένα του Gapminder στο http://www.gapminder.org/data/. Τα δεδομένα του ΑΕΠ είναι ο δείκτης "Income per person (GDP/capita, PPP$ inflation-adjusted)", πηγή <http://gapm.io/dgdppc>.

In [2]:
gdp_df = pd.read_csv('gapminder/income_per_person_gdppercapita_ppp_inflation_adjusted.csv')

gdp_df.sample(5, random_state=42)

,country,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
138,Palau,938,938,938,938,938,938,939,939,939,...,23k,23.5k,24k,24.5k,25k,25.5k,26.1k,26.6k,27.2k,27.7k
16,Bulgaria,1220,1220,1220,1220,1220,1220,1220,1220,1220,...,41.5k,42.4k,43.3k,44.2k,45.2k,46.1k,47.1k,48.1k,49.1k,50.2k
155,El Salvador,1090,1090,1090,1090,1090,1090,1090,1090,1090,...,12.2k,12.4k,12.7k,13k,13.2k,13.5k,13.8k,14.1k,14.4k,14.7k
96,Lao,966,967,967,967,967,967,967,967,968,...,14.7k,15k,15.3k,15.6k,16k,16.3k,16.6k,17k,17.4k,17.7k
68,Greece,1760,1760,1760,1760,1760,1760,1760,1760,1760,...,45.2k,46.1k,47.1k,48.1k,49.1k,50.2k,51.2k,52.3k,53.4k,54.6k


Παρατηρήστε ότι τα δεδομένα δεν είναι απλώς αριθμοί, καθώς μπορεί να έχουν την κατάληξη `k` (χιλιάδες). Γεκικότερα, τα δεδομένα από το Gapminder μπορεί να έχουν επίσης την κατάληξη `M` (εκατομμύρια) and `B` (δισεκατομμύρια).

Θα γράψουμε μια συνάρτηση για να μετατρέψουμε τα δεδομένα σε απλούς αριθμούς, χωρίς τις καταλήξεις.

In [3]:
suf2num_conv = {
    '' : 1,
    'k': 1e3,
    'm': 1e6,
    'b': 1e9
}

def suf2num(s):
    if s.dtype != object:
        return s
    n_s = s.str.extract(r'([\d\.]+)([kKmMbB]*)', expand=False)
    n_s[0] = n_s[0].astype(float)
    n_s[1] = n_s[1].str.lower()
    n_s['mul'] = n_s[1].map(lambda x : suf2num_conv.get(x, 1))
    return n_s[0] * n_s['mul']

suf2num(gdp_df['1800'])

0       674
1       691
2       746
3      1340
4      1120
       ... 
190    1570
191     981
192    1760
193     741
194     972
Name: 1800, Length: 195, dtype: int64

Θέλουμε να εφαρμόσουμε την μετατροπή σε όλες τις στήλες εκτός από την πρώτη. 

Για να το κάνουμε αυτό, θα εξάγουμε τη στήλη, θα μετατρέψουμε τις υπόλοιπες, και θα βάλουμε πίσω στη θέση της την στήλη που βγάλαμε.

In [4]:
gdp_countries = gdp_df.pop('country')
gdp_df = gdp_df.apply(suf2num, axis=0)
gdp_df['country'] = gdp_countries

Τα δεδομένα περιέχουν και προβολές στο μέλλον, τις οποίες θα τις αφαιρέσουμε (τη στιγμή που γράφονται αυτά οι προβλέψεις αφορού το 2021 και μετά).

In [5]:
gdp_df = gdp_df.drop([str(x) for x in range(2021, 2050+1)], axis=1)
gdp_df

,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,country
0,674,674,674,674,674,674,674,674,674,674,...,2080.0,2120.0,2100.0,2070.0,2060.0,2060.0,2030.0,2070.0,1920.0,Afghanistan
1,691,693,697,700,702,705,709,712,716,718,...,8040.0,8140.0,8240.0,8040.0,7570.0,7310.0,6930.0,6670.0,6220.0,Angola
2,746,746,746,746,746,747,747,747,747,747,...,11300.0,11400.0,11600.0,11900.0,12300.0,12800.0,13400.0,13700.0,13300.0,Albania
3,1340,1340,1340,1350,1350,1350,1350,1360,1360,1360,...,46900.0,48900.0,50200.0,52100.0,53900.0,55600.0,57600.0,59600.0,61600.0,Andorra
4,1120,1120,1120,1130,1130,1140,1140,1150,1150,1150,...,57400.0,59900.0,62400.0,65200.0,66500.0,67200.0,67000.0,67100.0,61300.0,United Arab Emirates
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,1570,1570,1570,1570,1570,1570,1570,1570,1570,1570,...,5910.0,5830.0,5790.0,6000.0,6450.0,6490.0,6330.0,6520.0,6260.0,Samoa
191,981,983,986,988,992,994,997,1000,1000,1010,...,4690.0,4790.0,4660.0,3270.0,2880.0,2670.0,2630.0,2610.0,2420.0,Yemen
192,1760,1740,1720,1700,1680,1590,1590,1730,1510,1480,...,12700.0,12900.0,12900.0,12800.0,12700.0,12700.0,12600.0,12500.0,11400.0,South Africa
193,741,743,746,747,749,750,752,755,756,758,...,3340.0,3400.0,3450.0,3440.0,3470.0,3490.0,3520.0,3470.0,3250.0,Zambia


Βρίσκουμε το ελάχιστο και το μέγιστο κατακεφαλήν ΑΕΠ για όλες τις χώρες. Θα χρειαστούμε αυτές τις τιμές αργότερα για να ρυθμίσουμε τα όρια του διαγράμματός μας.

In [6]:
min_gdp = gdp_df.iloc[:,:-1].min().min() # minimum per rows, then minimum across rows
max_gdp = gdp_df.iloc[:,:-1].max().max() # maximum per rows, then maximum across rows
print(min_gdp, max_gdp)

222.0 157000.0


Θέλουμε να ομαδοποιήσουμε τις χώρες σύμφωνα με τις γεωγραφικές περιοχές τους. Οι γεωγραφικές περιοχές είναι από το <https://www.gapminder.org/fw/four-regions/> και συγκεκριμένα το <http://gapm.io/datageo>. Αφού διαβάσουμε τις περιοχές, τις ενώνουμε με τα στοιχεία του ΑΕΠ χρησιμοποιώντας τη στήλη `country`.

In [7]:
geographical_regions_df = pd.read_excel('gapminder/Data Geographies - v2 - by Gapminder.xlsx',
                                        sheet_name='list-of-countries-etc')[['name', 'six_regions']]

all_df = pd.merge(gdp_df, geographical_regions_df, left_on='country', right_on='name')
all_df.sample(5, random_state=42)

,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,...,2014,2015,2016,2017,2018,2019,2020,country,name,six_regions
45,841,843,846,849,852,854,856,860,862,865,...,4240.0,4490.0,4710.0,4890.0,5210.0,5540.0,5330.0,Djibouti,Djibouti,middle_east_north_africa
137,938,938,938,938,938,938,939,939,939,939,...,17100.0,18400.0,18300.0,17800.0,18500.0,17600.0,15600.0,Palau,Palau,east_asia_pacific
76,1400,1400,1400,1400,1400,1400,1400,1400,1400,1400,...,26500.0,27600.0,28300.0,29600.0,31200.0,32600.0,31000.0,Hungary,Hungary,europe_central_asia
144,1230,1230,1240,1240,1250,1250,1250,1260,1260,1270,...,95600.0,96000.0,95600.0,91700.0,91000.0,90000.0,87500.0,Qatar,Qatar,middle_east_north_africa
113,674,674,674,674,674,674,674,674,674,674,...,2080.0,2140.0,2200.0,2250.0,2280.0,2320.0,2210.0,Mali,Mali,sub_saharan_africa


Στη συνέχεια παίρνουμε τα στοιχεία για το προσδόκιμο ζωής. Αυτός είναι ο δείκτης "Life expectancy (years)", διαθέσιμος στο <http://gapm.io/ilex>.

In [8]:
lex_df = pd.read_csv('gapminder/life_expectancy_years.csv')

lex_df.sample(5, random_state=42)

,country,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
138,Palau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,Bulgaria,35.8,35.8,35.9,35.9,36.0,36.0,36.1,36.1,36.1,...,82.4,82.5,82.6,82.7,82.8,82.9,83.0,83.1,83.2,83.3
155,El Salvador,28.7,28.7,28.7,28.7,28.7,28.7,28.7,28.7,28.7,...,87.8,88.0,88.1,88.2,88.3,88.4,88.5,88.6,88.8,88.9
96,Lao,31.9,31.9,31.9,31.9,31.9,31.9,31.9,31.9,31.9,...,81.1,81.3,81.4,81.5,81.7,81.8,82.0,82.1,82.2,82.4
68,Greece,36.6,36.6,36.6,36.6,36.6,36.6,36.6,36.6,36.6,...,89.4,89.5,89.6,89.7,89.8,89.9,90.0,90.2,90.3,90.4


Βρίσκουμε και πάλι την ελάχιστη και τη μέγιστη τιμή, που θα τις χρειαστούμε για την προσαρμογή των ορίων του διαγράμματός μας.

In [9]:
min_lex = lex_df.iloc[:,1:].min().min()
max_lex = lex_df.iloc[:,1:].max().max()
print(min_lex, max_lex)

1.01 94.4


Ενώνουμε τα δεδομένα του προσδόκιμου ζωής με το `DataFrame` `all_df`, το οποίο σιγά-σιγά θα μεγαλώνει ώστε να περιέχει *όλα* μας τα δεδομένα. Επειδή υπάρχουν στήλες με τα ίδια ονόματα στα δύο `DataFrame` που ενώνουμε, θα δώσουμε εμείς τις δικές μας καταλήξεις για να έχουμε καλύτερα ονόματα από αυτά με τις καταλήξεις `_x` και `_y` που θα έβαζε το pandas μόνο του.

In [10]:
all_df = pd.merge(all_df, 
                  lex_df, 
                  on='country',
                  suffixes=("_gdp", "_lex"))

In [11]:
all_df.sample(5, random_state=42)

,1800_gdp,1801_gdp,1802_gdp,1803_gdp,1804_gdp,1805_gdp,1806_gdp,1807_gdp,1808_gdp,1809_gdp,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
45,841,843,846,849,852,854,856,860,862,865,...,78.9,79.0,79.2,79.3,79.5,79.6,79.8,79.9,80.0,80.2
137,938,938,938,938,938,938,939,939,939,939,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,1400,1400,1400,1400,1400,1400,1400,1400,1400,1400,...,85.9,86.0,86.1,86.2,86.3,86.4,86.5,86.6,86.7,86.8
144,1230,1230,1240,1240,1250,1250,1250,1260,1260,1270,...,84.8,84.9,85.0,85.1,85.2,85.3,85.4,85.5,85.6,85.7
113,674,674,674,674,674,674,674,674,674,674,...,74.3,74.5,74.5,74.7,74.8,74.9,75.0,75.1,75.2,75.3


Τέλος θα πρέπει να πάρουμε τα δεδομένα για τον πληθυσμό κάθε χώρας. Αυτός είναι ο δείκτης "Population, total" διαθέσιμος στο <http://gapm.io/dpop>. 

In [12]:
pop_df = pd.read_csv('gapminder/population_total.csv')
pop_df

,country,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,Afghanistan,3.28M,3.28M,3.28M,3.28M,3.28M,3.28M,3.28M,3.28M,3.28M,...,76.6M,76.4M,76.3M,76.1M,76M,75.8M,75.6M,75.4M,75.2M,74.9M
1,Angola,1.57M,1.57M,1.57M,1.57M,1.57M,1.57M,1.57M,1.57M,1.57M,...,168M,170M,172M,175M,177M,179M,182M,184M,186M,188M
2,Albania,400k,402k,404k,405k,407k,409k,411k,413k,414k,...,1.33M,1.3M,1.27M,1.25M,1.22M,1.19M,1.17M,1.14M,1.11M,1.09M
3,Andorra,2650,2650,2650,2650,2650,2650,2650,2650,2650,...,63k,62.9k,62.9k,62.8k,62.7k,62.7k,62.6k,62.5k,62.5k,62.4k
4,United Arab Emirates,40.2k,40.2k,40.2k,40.2k,40.2k,40.2k,40.2k,40.2k,40.2k,...,12.3M,12.4M,12.5M,12.5M,12.6M,12.7M,12.7M,12.8M,12.8M,12.9M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,Samoa,47.3k,47.3k,47.3k,47.3k,47.3k,47.3k,47.3k,47.2k,47.2k,...,315k,314k,314k,314k,313k,313k,312k,312k,311k,310k
193,Yemen,2.59M,2.59M,2.59M,2.59M,2.59M,2.59M,2.59M,2.59M,2.59M,...,54.5M,54.4M,54.3M,54.1M,54M,53.8M,53.7M,53.5M,53.4M,53.2M
194,South Africa,1.45M,1.45M,1.46M,1.46M,1.47M,1.47M,1.48M,1.49M,1.49M,...,79.8M,79.8M,79.7M,79.7M,79.6M,79.5M,79.5M,79.4M,79.3M,79.2M
195,Zambia,747k,758k,770k,782k,794k,806k,818k,831k,843k,...,74.4M,75.2M,76M,76.8M,77.6M,78.4M,79.2M,80M,80.8M,81.5M


Όπως και πριν, θα βγάλουμε τη στήλη `country`, θα κάνουμε τη μετατροπή των δεδομένων σε απλούς αριθμούς χωρίς καταλήξεις, και θα ξαναβάλουμε τη στήλη πίσω στο `DataFrame`.

In [13]:
pop_countries = pop_df.pop('country')
pop_df = pop_df.apply(suf2num, axis=0)
pop_df['country'] = pop_countries
pop_df

,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,...,2092,2093,2094,2095,2096,2097,2098,2099,2100,country
0,3280000.0,3280000.0,3280000.0,3280000.0,3280000.0,3280000.0,3280000.0,3280000.0,3280000.0,3280000.0,...,76400000.0,76300000.0,76100000.0,76000000.0,75800000.0,75600000.0,75400000.0,75200000.0,74900000.0,Afghanistan
1,1570000.0,1570000.0,1570000.0,1570000.0,1570000.0,1570000.0,1570000.0,1570000.0,1570000.0,1570000.0,...,170000000.0,172000000.0,175000000.0,177000000.0,179000000.0,182000000.0,184000000.0,186000000.0,188000000.0,Angola
2,400000.0,402000.0,404000.0,405000.0,407000.0,409000.0,411000.0,413000.0,414000.0,416000.0,...,1300000.0,1270000.0,1250000.0,1220000.0,1190000.0,1170000.0,1140000.0,1110000.0,1090000.0,Albania
3,2650.0,2650.0,2650.0,2650.0,2650.0,2650.0,2650.0,2650.0,2650.0,2650.0,...,62900.0,62900.0,62800.0,62700.0,62700.0,62600.0,62500.0,62500.0,62400.0,Andorra
4,40200.0,40200.0,40200.0,40200.0,40200.0,40200.0,40200.0,40200.0,40200.0,40200.0,...,12400000.0,12500000.0,12500000.0,12600000.0,12700000.0,12700000.0,12800000.0,12800000.0,12900000.0,United Arab Emirates
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,47300.0,47300.0,47300.0,47300.0,47300.0,47300.0,47300.0,47200.0,47200.0,47200.0,...,314000.0,314000.0,314000.0,313000.0,313000.0,312000.0,312000.0,311000.0,310000.0,Samoa
193,2590000.0,2590000.0,2590000.0,2590000.0,2590000.0,2590000.0,2590000.0,2590000.0,2590000.0,2590000.0,...,54400000.0,54300000.0,54100000.0,54000000.0,53800000.0,53700000.0,53500000.0,53400000.0,53200000.0,Yemen
194,1450000.0,1450000.0,1460000.0,1460000.0,1470000.0,1470000.0,1480000.0,1490000.0,1490000.0,1500000.0,...,79800000.0,79700000.0,79700000.0,79600000.0,79500000.0,79500000.0,79400000.0,79300000.0,79200000.0,South Africa
195,747000.0,758000.0,770000.0,782000.0,794000.0,806000.0,818000.0,831000.0,843000.0,856000.0,...,75200000.0,76000000.0,76800000.0,77600000.0,78400000.0,79200000.0,80000000.0,80800000.0,81500000.0,Zambia


Θα αφαιρέσουμε τις προβολές.

In [14]:
pop_df = pop_df.drop([str(x) for x in range(2021, 2100+1)], axis=1)
pop_df

,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,country
0,3280000.0,3280000.0,3280000.0,3280000.0,3280000.0,3280000.0,3280000.0,3280000.0,3280000.0,3280000.0,...,31200000.0,32300000.0,33400000.0,34400000.0,35400000.0,36300000.0,37200000.0,38000000.0,38900000.0,Afghanistan
1,1570000.0,1570000.0,1570000.0,1570000.0,1570000.0,1570000.0,1570000.0,1570000.0,1570000.0,1570000.0,...,25100000.0,26000000.0,26900000.0,27900000.0,28800000.0,29800000.0,30800000.0,31800000.0,32900000.0,Angola
2,400000.0,402000.0,404000.0,405000.0,407000.0,409000.0,411000.0,413000.0,414000.0,416000.0,...,2910000.0,2900000.0,2900000.0,2890000.0,2890000.0,2880000.0,2880000.0,2880000.0,2880000.0,Albania
3,2650.0,2650.0,2650.0,2650.0,2650.0,2650.0,2650.0,2650.0,2650.0,2650.0,...,82400.0,80800.0,79200.0,78000.0,77300.0,77000.0,77000.0,77100.0,77300.0,Andorra
4,40200.0,40200.0,40200.0,40200.0,40200.0,40200.0,40200.0,40200.0,40200.0,40200.0,...,9140000.0,9200000.0,9210000.0,9260000.0,9360000.0,9490000.0,9630000.0,9770000.0,9890000.0,United Arab Emirates
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,47300.0,47300.0,47300.0,47300.0,47300.0,47300.0,47300.0,47200.0,47200.0,47200.0,...,189000.0,191000.0,192000.0,194000.0,195000.0,195000.0,196000.0,197000.0,198000.0,Samoa
193,2590000.0,2590000.0,2590000.0,2590000.0,2590000.0,2590000.0,2590000.0,2590000.0,2590000.0,2590000.0,...,24500000.0,25100000.0,25800000.0,26500000.0,27200000.0,27800000.0,28500000.0,29200000.0,29800000.0,Yemen
194,1450000.0,1450000.0,1460000.0,1460000.0,1470000.0,1470000.0,1480000.0,1490000.0,1490000.0,1500000.0,...,52800000.0,53700000.0,54500000.0,55400000.0,56200000.0,57000000.0,57800000.0,58600000.0,59300000.0,South Africa
195,747000.0,758000.0,770000.0,782000.0,794000.0,806000.0,818000.0,831000.0,843000.0,856000.0,...,14500000.0,14900000.0,15400000.0,15900000.0,16400000.0,16900000.0,17400000.0,17900000.0,18400000.0,Zambia


Τα ενώνουμε όλα μαζί.

In [15]:
all_df = pd.merge(all_df, 
                  pop_df, 
                  on='country')   
all_df.sample(10, random_state=42)

,1800_gdp,1801_gdp,1802_gdp,1803_gdp,1804_gdp,1805_gdp,1806_gdp,1807_gdp,1808_gdp,1809_gdp,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
45,841,843,846,849,852,854,856,860,862,865,...,854000.0,868000.0,883000.0,899000.0,914000.0,929000.0,944000.0,959000.0,974000.0,988000.0
137,938,938,938,938,938,938,939,939,939,939,...,17700.0,17600.0,17600.0,17600.0,17700.0,17700.0,17800.0,17900.0,18000.0,18100.0
76,1400,1400,1400,1400,1400,1400,1400,1400,1400,1400,...,9900000.0,9860000.0,9830000.0,9800000.0,9780000.0,9750000.0,9730000.0,9710000.0,9680000.0,9660000.0
144,1230,1230,1240,1240,1250,1250,1250,1260,1260,1270,...,2040000.0,2200000.0,2340000.0,2460000.0,2570000.0,2650000.0,2720000.0,2780000.0,2830000.0,2880000.0
113,674,674,674,674,674,674,674,674,674,674,...,15500000.0,16000000.0,16400000.0,16900000.0,17400000.0,18000000.0,18500000.0,19100000.0,19700000.0,20300000.0
152,406,406,406,406,406,406,406,406,406,406,...,542000.0,556000.0,571000.0,587000.0,603000.0,619000.0,636000.0,653000.0,670000.0,687000.0
15,979,979,979,979,979,979,979,979,979,978,...,149000000.0,151000000.0,153000000.0,155000000.0,156000000.0,158000000.0,160000000.0,161000000.0,163000000.0,165000000.0
18,1620,1620,1620,1620,1620,1620,1620,1620,1620,1620,...,360000.0,364000.0,367000.0,371000.0,374000.0,378000.0,382000.0,386000.0,389000.0,393000.0
16,1220,1220,1220,1220,1220,1220,1220,1220,1220,1220,...,7380000.0,7330000.0,7290000.0,7250000.0,7200000.0,7150000.0,7100000.0,7050000.0,7000000.0,6950000.0
9,2070,2070,2080,2090,2100,2110,2120,2120,2130,2140,...,8450000.0,8500000.0,8560000.0,8620000.0,8680000.0,8750000.0,8820000.0,8890000.0,8960000.0,9010000.0


Υπάρχει ένα λεπτό σημείο που πρέπει να φροντίσουμε. Το διάγραμμα να σχεδιαστεί μέσω JavaScript. Άρα τα ονόματα των στηλών του `DataFrame` θα πρέπει να είναι κατανοητά από την JavaScript. Τα ονόματα που στηλών που ξεκινούν με αριθμούς όμως δεν θα γίνουν αποδεκτά από την JavaScript, επομένως θα πρέπει να τα αναποδογυρίσουμε ώστε το `2020_gdp` ναι γίνει `gdp_2020`, το `2020_lex` να γίνει `lex_2020`, το `2020` να γίνει `pop_2020`, κ.λπ. για όλες τις χρονιές.

Ενόσω το κάνουμε αυτό, θα δημιουργήσουμε και μία ακόμα στήλη που θα χρησιμοποιήσουμε για το μέγεθος με το οποίο θα απεικονίσουμε κάθε χώρα. Το μέγεθος θα είναι ανάλογο με τον πληθυσμό, οπότε θα προσθέσουμε μια στήλη `size_x` (όπου `x` είναι η χρονιά). Επειδή το μέγεθος θα ορίζεται με βάση την ακτίνα του κύκλου για κάθε χώρα, θα πρέπει να πάρουμε την τετραγωνική ρίζα του πληθυσμού δια π.

Όλα αυτά θα γίνουν πιο γρήγορα αν βάλουμε τις νέες στήλες `size_x` σε ένα νεο `DataFrame` και καλέσουμε την `concat()`.

In [16]:
size_cols = {}
for column in all_df.columns:
    col = str(column) # needed for those columns whose name is a number (year)
    col_match = re.match(r'(\d+)(_(.+))?', col)
    if col_match:
        if col_match.lastindex > 1:
            new_name = col_match.group(3) + '_' + col_match.group(1)
            all_df.rename(columns={column: new_name},
                          inplace=True)
        else:
            new_name = 'pop_' + col_match.group(1)
            all_df.rename(columns={column: new_name},
                          inplace=True)
            sizes = 0.003 * np.sqrt(all_df[new_name] / np.pi)
            size_cols['size_' + col_match.group(1)] = sizes

all_df = pd.concat([all_df, pd.DataFrame(size_cols)], axis=1)
all_df

,gdp_1800,gdp_1801,gdp_1802,gdp_1803,gdp_1804,gdp_1805,gdp_1806,gdp_1807,gdp_1808,gdp_1809,...,size_2011,size_2012,size_2013,size_2014,size_2015,size_2016,size_2017,size_2018,size_2019,size_2020
0,674,674,674,674,674,674,674,674,674,674,...,9.286019,9.454175,9.619391,9.781817,9.927172,10.070428,10.197639,10.323282,10.433695,10.556528
1,691,693,697,700,702,705,709,712,716,718,...,8.326337,8.479753,8.630441,8.778543,8.940224,9.083277,9.239627,9.393375,9.544647,9.708324
2,746,746,746,746,746,747,747,747,747,747,...,2.897211,2.887306,2.882341,2.882341,2.877367,2.877367,2.872384,2.872384,2.872384,2.872384
3,1340,1340,1340,1350,1350,1350,1350,1360,1360,1360,...,0.489676,0.485859,0.481118,0.476331,0.472709,0.470583,0.469669,0.469669,0.469974,0.470583
4,1120,1120,1120,1130,1130,1140,1140,1150,1150,1150,...,5.063582,5.117047,5.133815,5.136605,5.150529,5.178265,5.214101,5.252420,5.290462,5.322853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,1570,1570,1570,1570,1570,1570,1570,1570,1570,1570,...,0.731926,0.735830,0.739713,0.741646,0.745499,0.747418,0.747418,0.749332,0.751241,0.753146
189,981,983,986,988,992,994,997,1000,1000,1010,...,8.257238,8.377788,8.479753,8.597183,8.713031,8.827359,8.924188,9.035844,9.146138,9.239627
190,1760,1740,1720,1700,1680,1590,1590,1730,1510,1480,...,12.205287,12.298815,12.403192,12.495239,12.597988,12.688622,12.778614,12.867976,12.956722,13.033878
191,741,743,746,747,749,750,752,755,756,758,...,6.333012,6.445110,6.533403,6.642119,6.749085,6.854381,6.958084,7.060264,7.160986,7.260311


Κάθε χώρα θα εμφανιστεί με χρώμα που αντιστοιχεί στη γεωγραφική της περιοχή. Για να γίνει αυτό, θα πρέπει να πάρουμε έναν αριθμητικό κωδικό για κάθε χώρα.

In [17]:
all_df['Region Cat'] = all_df['six_regions'].astype('category')

Το διάγραμμα θα δείχνει το προσδόκιμο ζωής, κατακεφαλήν ΑΕΠ, και πληθυσμό για κάθε χρόνο, τον οποίο θα επιλέγουμε αλληλεπιδραστικά. Θα προσθέσουμε στο `DataFrame` μας:

* μια στήλη `x` με το κατακεφαλήν ΑΕΠ για την επιλεγμένη χρονιά

* μια στήλη `y` με το προσδόκιμο ζωής για την επιλεγμένη χρονιά

* μια στήλη `pop` με τον πληθυσμό για την επιλεγμένη χρονιά

* μια στήλη `size` με το μέγεθος της χώρας για την επιλεγμένη χρονιά

Επίσης θα συγκεντρώσουμε τα χρώματα της κάθε χώρας. Αυτά θα προκύψουν από τη χρωματική παλέτα (`Spectral6` αφού έχουμε έξι περιοχές) και τον κωδικό γεωγραφικής περιοχής κάθε χώρας.

In [18]:
from bokeh.palettes import Spectral6

all_df['x'] = all_df['gdp_2020']
all_df['y'] = all_df['lex_2020']
all_df['pop'] = all_df['pop_2020']

all_df['size'] = all_df['size_2020']
all_df['colors'] = all_df['Region Cat'].cat.codes.map(lambda x: Spectral6[x])

Τα υπόλοιπα είναι θέμα Bokeh και JavaScript.

In [19]:
import bokeh.plotting as bk

from bokeh.io import show, output_notebook

from bokeh.models import (ColumnDataSource,
                          HoverTool, 
                          BoxZoomTool,
                          ResetTool,
                          PanTool,
                          WheelZoomTool,
                          CustomJS,
                          Circle,
                          Text,
                          NumeralTickFormatter)

from bokeh.models.widgets import Slider

from bokeh.layouts import column, gridplot

dim = 1000 # width and length

source = ColumnDataSource(all_df)

hover = HoverTool(tooltips=[
        ("Country Name", "@country"),
        ("Population", "@pop")
        ]
    )

min_x = 100 * (min_gdp // 100)
max_x = 1000 * (max_gdp // 1000) + 1000

min_y = 10 * (min_lex // 10)
max_y = 100 * (max_lex // 100) + 100

tools = [
    hover,
    WheelZoomTool(),
    PanTool(),
    BoxZoomTool(),
    ResetTool()
]

p = bk.figure(tools=tools,
              x_axis_type="log",
              x_axis_label="Income per person "
              "(GDP/capita, PPP$ inflation-adjusted)",
              y_axis_label="Life expectancy (years)",
              x_range=(min_x, max_x), 
              y_range=(min_y, max_y),
              width=dim,
              height=dim)

p.xaxis[0].formatter = NumeralTickFormatter(format='0a')

text_x = 15000
text_y = 20

regions = all_df['Region Cat'].cat.categories
for i, region in enumerate(regions):
    p.add_glyph(Text(x=text_x, 
                     y=text_y, 
                     text=[region], 
                     text_font_size='10pt', 
                     text_color='#666666'))
    p.add_glyph(Circle(x=text_x - 2000, 
                       y=text_y + 1, 
                       fill_color=Spectral6[i], 
                       size=10, 
                       line_color=None, fill_alpha=0.8))
    text_y = text_y - 3

callback = CustomJS(args=dict(source=source), code="""
        var data = source.data;
        var v = cb_obj.value;
        var x = data['gdp_' + v];
        var y = data['lex_' + v];
        data['x'] = x;
        data['y'] = y;
        // As we do not have data for size and population
        // for all year, go to the nearest year for which
        // we do have data.
        if (!data['size_' + v]) {
          v = Math.ceil(v / 10) * 10;
        }
        data['size'] = data['size_' + v];
        data['pop'] = data['pop_' + v];
        source.change.emit();
        // source.trigger('change');
    """)

p.scatter(x='x', 
          y='y',
          size='size',
          fill_color='colors',
          fill_alpha=0.8,
          source=source)

slider = Slider(start=1800, end=2020, value=2020, step=1, title="Year", width=dim)
slider.js_on_change('value', callback)


bk.output_notebook()
bk.output_file('gdp_life_expectancy_gapminder.html')
#pg = gridplot([p, slider], ncols=1, sizing_mode='stretch_both')
#bk.show(pg)
bk.show(column(p, slider))

Loading BokehJS ...